# data set 
Foi pego no uci, convertido para csv com auxilio de script


In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from statistics import mean 
from collections import Counter
from sklearn.metrics import confusion_matrix


In [2]:
headers = list(pd.read_csv("EGG_EyeStateHeaders.csv") )

In [3]:
data = pd.read_csv("EEG_EyeState.csv", names=headers, index_col=False, skiprows = [0]) 
# Preview the first 5 lines of the loaded data 
data.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, data.eyeDetection, test_size=0.3)                                                    
X_train = X_train.drop("eyeDetection", axis=1)
X_test = X_test.drop("eyeDetection", axis=1)

In [5]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

           AF3       F7       F3      FC5       T7       P7       O1       O2  \
9796   4284.10  3984.62  4249.23  4100.00  4334.36  4595.90  4043.08  4605.64   
12914  4277.95  4003.08  4261.54  4123.08  4341.54  4618.97  4058.97  4608.21   
9451   4291.79  3988.21  4249.74  4102.56  4343.59  4621.54  4057.44  4596.92   
11527  4294.87  4017.44  4263.59  4136.92  4349.23  4626.67  4082.56  4609.23   
6605   4292.31  4002.05  4263.08  4120.51  4329.74  4614.87  4054.87  4615.90   
...        ...      ...      ...      ...      ...      ...      ...      ...   
4298   4345.13  4049.74  4278.46  4140.51  4332.82  4606.67  4053.33  4587.69   
5232   4423.08  4066.15  4298.46  4154.36  4350.26  4626.15  4085.13  4604.62   
10059  4304.62  4005.64  4271.28  4127.18  4345.64  4625.13  4057.95  4599.49   
9817   4281.03  3977.44  4251.28  4098.46  4328.21  4601.54  4048.21  4596.92   
1407   4286.15  3949.74  4250.26  4076.41  4323.08  4603.59  4093.85  4630.26   

            P8       T8    

Algoritmo diferente para a harmonica

In [6]:
from scipy.spatial import distance as dist_lib
def euclidean(instance1, instance2):
    return dist_lib.euclidean(instance1, instance2)

def canberra(instance1, instance2):
    return dist_lib.canberra(instance1, instance2)

def chebyshev(instance1, instance2):
    return dist_lib.chebyshev(instance1, instance2)

def correlation(instance1, instance2):
    return dist_lib.correlation(instance1, instance2)

def cosine(instance1, instance2):
    return dist_lib.cosine(instance1, instance2)

In [7]:
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance):
    """
    get_neighors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label),
    where:
    index    is the index from the training_set, 
    dist     is the distance between the test_instance 
             and the instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = []
    for index, row in training_set.iterrows():
        dist = distance(test_instance, row)
        distances.append((row, dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors


            
def vote_harmonic_weights(neighbors, all_results=True):
    class_counter = Counter()
    number_of_neighbors = len(neighbors)
    
    # para cada classe, somam-se os inversos de suas ordens
    for index in range(number_of_neighbors):
        class_counter[neighbors[index][2]] += 1/(index+1)
    labels, votes = zip(*class_counter.most_common())
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    if all_results:
        total = sum(class_counter.values(), 0.0)
        for key in class_counter:
             class_counter[key] /= total
        return winner, class_counter.most_common()
    else:
        return winner, votes4winner / sum(votes)
    
def knn_test_harmonic(trainset_data, trainset_labels, testset_data, knn_k, distance_function):
    print(testset_data.shape[0])
    count_row = 0
    df_prediction = pd.DataFrame(columns=['class'])
    for index, row in testset_data.iterrows():
        neighbors = get_neighbors(trainset_data, 
                                  trainset_labels, 
                                  row, 
                                  knn_k, 
                                  distance=distance_function)
        #print(vote_harmonic_weights(neighbors, all_results=True))
        predited_class = vote_harmonic_weights(neighbors, all_results=True)[0]
        df_prediction.loc[index] = predited_class
        count_row += 1
        if (count_row % 50 == 0):
            print(count_row, count_row/testset_data.shape[0])
    
    return df_prediction

In [8]:
weights = [#'uniform', 'distance', 
    'harmonic']

distances = ['canberra', 'chebyshev', 'correlation', 'euclidean','cosine' ] #'mahalanobis'
V = np.cov(X_train)

In [9]:
def cross_validation(knn_k, distance, weight_function):
    if (weight_function == 'harmonic'):
        kf = KFold(n_splits= 5000, shuffle=True)
        accs = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
                
            print("Treino: " + str(X_train.shape[0]) + ", fold_treino: " + str(fold_x_train.shape[0]) + ", fold_test: "+ str(fold_x_test.shape[0]))
            
            #Train the model using the training sets
            if (distance == 'canberra'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, canberra)
            elif (distance == 'chebyshev'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'correlation'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'euclidean'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'cosine'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
                
            acc = metrics.accuracy_score(fold_y_test, fold_y_pred['class'].astype('int32'))
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(y_test, y_pred)
            print(cm)
        
        return [knn_k, distance, weight_function, mean(accs)]
            
    else:
        #Create KNN Classifier
        if distance == 'mahalanobis':
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
        else:
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

        kf = KFold(n_splits= 5, shuffle=True)
        accs = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
            #Train the model using the training sets
            knn.fit(fold_x_train, fold_y_train)

            #Predict the response for test dataset
            fold_y_pred = knn.predict(fold_x_test)

            acc = metrics.accuracy_score(fold_y_test, fold_y_pred)
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(y_test, y_pred)
    
        return [knn_k, distance, weight_function, mean(accs)]

In [10]:
def test(knn_k, distance, weight_function):
    #Create KNN Classifier
    if distance == 'mahalanobis':
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
    else:
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

    #Train the model using the training sets
    knn.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = knn.predict(X_test)

    cm_test = confusion_matrix(y_test, y_pred)

    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [11]:
def select_bests_params(results):
    df_results = pd.DataFrame(results)
    return df_results.sort_values([3],  ascending=[0])

In [12]:
results = []
for weight_function in weights:
    for knn_k in range(1,5,2):
        for distance in distances:
            print("Função: {} - distancia: {} - K: {}".format(weight_function,distance, 5))
            results.append(cross_validation(knn_k, distance, weight_function))

ranking = select_bests_params(results)
print(ranking)

Função: harmonic - distancia: canberra - K: 5
Treino: 10486, fold_treino: 7288, fold_test: 1
1
Accuracy: 1.0


NameError: name 'y_pred' is not defined

In [ ]:
best_knn_k, best_distance, best_weight_function, best_avg_acc = list(ranking.head(1).itertuples(index=False, name=None))[0]

In [ ]:
best_knn_k, best_distance, best_weight_function

In [ ]:
test(best_knn_k, best_distance, best_weight_function)

In [ ]:
type(y_test)

#acc = metrics.accuracy_score(fold_y_test, fold_y_pred)